# **Catégorisez automatiquement des questions**

## partie 1/8 : analyse exploratoire

### <br> Notebook d’exploration et de pré-traitement des questions, comprenant une analyse univariée et multivariée, un nettoyage des questions, un feature engineering de type bag of words avec réduction de dimension (du vocabulaire et des tags) 

<br>


## 1 Objectifs, imports


### 1.1 Contexte


In [ ]:
# Nous avons téléchargé 50 000 questions posées sur stack overflow,
# ainsi que les tags associés.

# Objectifs de notebook :

# Comprendre nos données (forme, structure)
# premier nettoyage -> reduction dim
# Avoir une première représentation de nos données
# Rechercher des patterns
# se faire une idée de la difficulté de la tâche (prédiction de tags)
# (encore et surtout) : transformations en vue de reduction dim

# => traitement de strings avec nltk, regex, etc..
# tester spacy

# Dans le cadre de ce projet :
# Nous avons en input un fichier csv contenant 50 000 titres, questions (body), tags, etc...
# Nous avons besoin en output de représentations spécifiques, pour nos différents modèles.

# Pour les modèles non-supervisé, il nous faut :
# - un bag of words pour la LDA (/algos similaires)
# - un TS-IDF pour la NMF
# Dans les deux cas, cela implique un cleaning "aggressif", visant à conserver uniquement
# les mots qui ont du sens pour ce projet (= qui aident la prédiction de tag par les modèles).
# encore et tjs, la reduction dim !

# / supervisés ?


### 1.2 Importation des librairies, réglages


In [ ]:
import os, sys, random
from zipfile import ZipFile
import numpy as np
import pandas as pd
from pandarallel import pandarallel

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
from PIL import Image

# NLP
from bs4 import BeautifulSoup
import re, string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from collections import Counter
# if necessary
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_md
import spacy

print('\nPython version ' + sys.version)
print('pandas version ' + pd.__version__)
print('sns version ' + sns.__version__)

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
sns.set(font_scale=1)

# Make sure we downloaded the models successfully
nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")

# Modify if necessary
num_cores = os.cpu_count()
print(f"\nNumber of CPU cores: {num_cores}")
pandarallel.initialize(progress_bar=False, nb_workers=6)


### 1.3 Fonctions


In [ ]:
# Principalement des fonctions pour l'affichage des graphiques pdt l'EDA

def quick_look(df, miss=True):
    """
    Display a quick overview of a DataFrame, including shape, head, tail, unique values, and duplicates.

    Args:
        df (pandas.DataFrame): The input DataFrame to inspect.
        check_missing (bool, optional): Whether to check and display missing values (default is True).

    The function provides a summary of the DataFrame, including its shape, the first and last rows, the count of unique values per column, and the number of duplicates.
    If `check_missing` is set to True, it also displays missing value information.
    """
    print(f'shape : {df.shape}')

    display(df.head())
    display(df.tail())

    print('uniques :')
    display(df.nunique())

    print('Doublons ? ', df.duplicated(keep='first').sum(), '\n')

    if miss:
        display(get_missing_values(df))


def lerp(a, b, t):
    """
    Linear interpolation between two values 'a' and 'b' at a parameter 't'.
    A very useful little function, used here to position annotations in plots.
    Got it coding with Radu :)

    Given two values 'a' and 'b', and a parameter 't',
    this function calculates the linear interpolation between 'a' and 'b' at 't'.

    Parameters:
    a (float or int): The start value.
    b (float or int): The end value.
    t (float): The interpolation parameter (typically in the range [0, 1], but can be outside).

    Returns:
    float or int: The interpolated value at parameter 't'.
    """
    return a + (b - a) * t


def generate_random_pastel_colors(n):
    """
    Generates a list of n random pastel colors, represented as RGBA tuples.

    Parameters:
    n (int): The number of pastel colors to generate.

    Returns:
    list: A list of RGBA tuples representing random pastel colors.

    Example:
    >>> generate_random_pastel_colors(2)
    [(0.749, 0.827, 0.886, 1.0), (0.886, 0.749, 0.827, 1.0)]
    """
    colors = []
    for _ in range(n):
        # Generate random pastels
        red = round(random.randint(150, 250) / 255.0, 3)
        green = round(random.randint(150, 250) / 255.0, 3)
        blue = round(random.randint(150, 250) / 255.0, 3)

        # Create an RGB color tuple and add it to the list
        color = (red,green,blue, 1.0)
        colors.append(color)

    return colors

print(generate_random_pastel_colors(2))


def get_missing_values(df):
    """Generates a DataFrame containing the count and proportion of missing values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
        pandas.DataFrame: A DataFrame with columns for the feature name, count of missing values,
        count of non-missing values, proportion of missing values, and data type for each feature.
    """
    # Count the missing values for each column
    missing = df.isna().sum()

    # Calculate the percentage of missing values
    percent_missing = df.isna().mean() * 100

    # Create a DataFrame to store the results
    missings_df = pd.DataFrame({
        'column_name': df.columns,
        'missing': missing,
        'present': df.shape[0] - missing,  # Count of non-missing values
        'percent_missing': percent_missing.round(2),  # Rounded to 2 decimal places
        'type': df.dtypes
    })

    # Sort the DataFrame by the count of missing values
    missings_df.sort_values('missing', inplace=True)

    return missings_df

# with pd.option_context('display.max_rows', 1000):
#   display(get_missing_values(df))


def hist_distrib(dataframe, feature, bins, decimal_places, density=True):
    """
    Visualize the empirical distribution of a numerical feature using a histogram.
    Calcul des principaux indicateurs de tendance centrale, dispersion et forme.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.
        bins (int): The number of bins for the histogram.
        decimal_places (int): The number of decimal places for rounding numeric values.
        density (bool, optional): Whether to display the histogram as a density plot (default is True).

    Returns:
        float: The skewness of the feature's distribution.

    The function generates a histogram of the feature, displays various statistics, and returns the skewness of the distribution.
    """
    # Calculate central tendencies and dispersion
    mode_value = round(dataframe[feature].mode()[0], decimal_places)
    mode_non_zero = "N/A"
    if (dataframe[feature] != 0).any():
        mode_non_zero = round(dataframe.loc[dataframe[feature] != 0, feature].mode()[0], decimal_places)
    median_value = round(dataframe[feature].median(), decimal_places)
    mean_value = round(dataframe[feature].mean(), decimal_places)

    # Calculate dispersion
    var_emp = round(dataframe[feature].var(ddof=0), decimal_places)
    coeff_var = round(dataframe[feature].std(ddof=0), decimal_places)

    # Calculate shape indicators
    skewness_value = round(dataframe[feature].skew(), 2)
    kurtosis_value = round(dataframe[feature].kurtosis(), 2)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 5))
    dataframe[feature].hist(density=density, bins=bins, ax=ax)

    # Adjust placement for annotations
    yt = plt.yticks()
    y_position = lerp(yt[0][0], yt[0][-1], 0.8)
    y_increment = y_position / 20
    xt = plt.xticks()
    x_position = lerp(xt[0][0], xt[0][-1], 0.7)

    # Add annotations with horizontal and vertical alignment
    annotation_fs = 13
    color = 'g'
    ax.annotate(f'Mode: {mode_value}', xy=(x_position, y_position), fontsize=annotation_fs,
                xytext=(x_position, y_position), color=color, ha='left', va='bottom')
    ax.annotate(f'Mode +: {mode_non_zero}', xy=(x_position, y_position - y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Median: {median_value}', xy=(x_position, y_position - 2 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 2 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Mean: {mean_value}', xy=(x_position, y_position - 3 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 3 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Var Emp: {var_emp}', xy=(x_position, y_position - 5 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 5 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Coeff Var: {coeff_var}', xy=(x_position, y_position - 6 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 6 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Skewness: {skewness_value}', xy=(x_position, y_position - 8 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 8 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Kurtosis: {kurtosis_value}', xy=(x_position, y_position - 9 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 9 * y_increment), color=color, ha='left', va='bottom')

    # Label the x-axis and y-axis
    ax.set_xlabel(feature, fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)

    # Show the plot
    plt.title(f'Distribution of {feature}', pad=20, fontsize=18)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

    return skewness_value


def boxplot_distrib(dataframe, feature):
    """
    Affiche un boxplot, pour visualiser les tendances centrales et la dispersion d'une variable.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.

    The function generates a box plot of the feature to display central tendencies (median and mean) and dispersion.
    """
    fig, ax = plt.subplots(figsize=(10, 4))

    medianprops = {'color':"blue"}
    meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}

    dataframe.boxplot(feature, vert=False, showfliers=False, medianprops=medianprops, patch_artist=True, showmeans=True, meanprops=meanprops)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


def courbe_lorenz(dataframe, feature):
    """
    Affiche une courbe de Lorenz, pour visualiser la concentration d'une variable
    Calcule l'indice de Gini
    Visualize a Lorenz curve to assess the concentration of a variable and calculate the Gini coefficient.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.

    The function generates a Lorenz curve to assess the concentration of the feature and calculates the Gini coefficient.
    """
    fig, ax = plt.subplots(figsize=(12, 5))
    values = dataframe.loc[dataframe[feature].notna(), feature].values
    # print(values)
    n = len(values)
    lorenz = np.cumsum(np.sort(values)) / values.sum()
    lorenz = np.append([0],lorenz) # La courbe de Lorenz commence à 0

    xaxis = np.linspace(0-1/n,1+1/n,n+1)
    #Il y a un segment de taille n pour chaque individu, plus 1 segment supplémentaire d'ordonnée 0.
    # #Le premier segment commence à 0-1/n, et le dernier termine à 1+1/n.
    plt.plot(xaxis,lorenz,drawstyle='steps-post')
    plt.plot(np.arange(2),[x for x in np.arange(2)])
    # calcul de l'indice de Gini
    AUC = (lorenz.sum() -lorenz[-1]/2 -lorenz[0]/2)/n # Surface sous la courbe de Lorenz. Le premier segment (lorenz[0]) est à moitié en dessous de 0, on le coupe donc en 2, on fait de même pour le dernier segment lorenz[-1] qui est à moitié au dessus de 1.
    S = 0.5 - AUC # surface entre la première bissectrice et le courbe de Lorenz
    gini = 2*S
    plt.annotate('gini =  ' + str(round(gini, 2)), xy = (0.04, 0.88), fontsize = 13, xytext = (0.04, 0.88), color = 'g')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


def graphs_analyse_uni(dataframe, feature, bins=50, r=5, density=True):
    """
    Affiche histogramme + boxplot + courbe de Lorenz

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to analyze.
        bins (int, optional): The number of bins for the histogram (default is 50).
        decimal_places (int, optional): The number of decimal places for rounding numeric values (default is 5).
        density (bool, optional): Whether to display the histogram as a density plot (default is True).

    The function generates and displays an analysis of the given numerical feature, including an histogram, a box plot, and a Lorenz curve.
    """
    hist_distrib(dataframe, feature, bins, r)
    boxplot_distrib(dataframe, feature)
    courbe_lorenz(dataframe, feature)


def shape_head(df, nb_rows=5):
    """
    Affiche les dimensions et les premières lignes dùun dataframe
    Display the dimensions and the first rows of a DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame to display.
        nb_rows (int, optional): The number of rows to display (default is 5, max is 60).

    The function prints the dimensions of the DataFrame and displays the first few rows.
    """
    print(df.shape)
    display(df.head(nb_rows))


def doughnut(df, feature, title, width=10, height=10):
    """
    Affiche la répartition d'une feature sous forme de diagramme circulaire
    Display the distribution of a feature as a doughnut chart.
    Les couleurs sont aléatoires.

    Args:
        df (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the feature to visualize.
        title (str): The title for the doughnut chart.
        width (int, optional): The width of the chart (default is 10).
        height (int, optional): The height of the chart (default is 10).

    The function creates a doughnut chart to visualize the distribution of the specified feature.
    If you don't like the colors, try running it again :)
    """
    colors = generate_random_pastel_colors(20)

    grouped_df = df.groupby(feature).size().to_frame("count_per_type").reset_index()
    pie = grouped_df.set_index(feature).copy()

    fig, ax = plt.subplots(figsize=(width, height))

    patches, texts, autotexts = plt.pie(x=pie['count_per_type'], autopct='%1.1f%%',
        startangle=-30, labels=pie.index, textprops={'fontsize':11, 'color':'#000'},
        labeldistance=1.25, pctdistance=0.85, colors=colors)

    plt.title(
    label=title,
    fontdict={"fontsize":17},
    pad=20
    )

    for text in texts:
        # text.set_fontweight('bold')
        text.set_horizontalalignment('center')

    # Customize percent labels
    for autotext in autotexts:
        autotext.set_horizontalalignment('center')
        autotext.set_fontstyle('italic')
        autotext.set_fontsize('10')

    #draw circle
    centre_circle = plt.Circle((0,0),0.7,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    plt.show()


def get_non_null_values(df):
    """
    Génère un dataframe contenant le nombre et la proportion de non-null (non-zero) valeurs pour chaque feature
    Generate a DataFrame containing the count and proportion of non-null (non-zero) values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    The function calculates and returns a DataFrame with the count and percentage of non-null (non-zero) values for each feature.
    """
    non_null_counts = df.ne(0).sum()
    percent_non_null = (non_null_counts / df.shape[0]) * 100
    non_null_values_df = pd.DataFrame({'column_name': df.columns,
                                       'non_null_count': non_null_counts,
                                       'percent_non_null': percent_non_null.round(2),
                                       'type': df.dtypes})
    non_null_values_df.sort_values('non_null_count', inplace=True)
    return non_null_values_df


def get_colors(n=7):
    """
    Generate a list of random colors from multiple colormaps.

    Args:
        n (int, optional): The number of colors to sample from each colormap (default is 7).

    Returns:
        list: A list of random colors sampled from different colormaps.
    """
    num_colors_per_colormap = n
    colormaps = [plt.cm.Pastel2, plt.cm.Set1, plt.cm.Paired]
    all_colors = []

    for colormap in colormaps:
        colors = colormap(np.linspace(0, 1, num_colors_per_colormap))
        all_colors.extend(colors)

    np.random.shuffle(all_colors)

    return all_colors


## 2 EDA et preprocessing des questions


### 2.1 Importation des données brutes


In [ ]:
# Données compressées sinon on dépasse la limite /objet de Github (50Mb)

# path to the zip file
zip_file_path = './../data/raw_data/QueryResults.zip'

# directory where you want to extract the contents
extract_to_dir = './../data/raw_data'

# Open the zip file
with ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents into the specified directory
    zip_ref.extractall(extract_to_dir)

# L'encodage est bien UTF-8 (vérifié en ouvrant le .csv ds vscode)
raw_data = pd.read_csv('./../data/raw_data/QueryResults.csv', sep=',')

quick_look(raw_data)


In [ ]:
# Pas de valeurs manquantes, à part FavoriteCount (pas important)

# Seulement 2 titres identiques / 50 000 lignes
# id est bien une clé primaire
# body aussi, pas de questions en doublon (identiques)

# On a environ 48 000 sets de tags différents, vérifier + tard combien de tags uniques

# Les types semblent corrects,
# à part les dates bien sûr

raw_data['CreationDate'] = pd.to_datetime(raw_data['CreationDate'])
# Après je pense qu'on n'utilisera jamais ces dates... Juste au cas où.

raw_data.describe()

# Avec nos critères (cf requete sql, fin ntbk2), il a fallu retourner jusqu'à mai 2011
# pour avoir 50 000 questions. On retrouve ici le favoriteCount très bas, proche de 0.
# On a une moyenne pour le score (environ 50), le nv de vues (66 000) et de réponses (>5)
# dans notre corpus.


In [ ]:
raw_questions_tags = raw_data[['Title', 'Body', 'Tags']].copy()

# Rename columns
raw_questions_tags = raw_questions_tags.rename(columns={'Title': 'title',
                                                        'Body': 'body',
                                                        'Tags': 'tags'})


### 2.2 Suppression des tags html


In [ ]:
raw_questions_tags['body'] = raw_questions_tags['body'].parallel_apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())

quick_look(raw_questions_tags)

# environ 25s sans pandarallel -> 5-6s sur 6 cores
# pandarallel adopté !


In [ ]:
# Je ne vois pas de balises dans les titres
# mais j'avoue je n'ai pas lu les 50 000 titres, donc au cas où :

raw_questions_tags['title'] = raw_questions_tags['title'].parallel_apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())

# Le warning n'a pas d'importance ici : du texte qui ne contient pas de tags html
# n'est pas modifié par BeautifulSoup.get_text()

# J'hésitais à concaténer title + body ici, mais on testera aussi les modeles seulement sur les titres
# pour avoir une baseline. On va donc conserver les 2 features, et les traiter en parallèle.


### 2.3 Tokenisation, majuscules, ponctuation


In [ ]:
# Après qq tests, le + efficace ici semble d'appliquer d'abord la fonction word_tokenize(),
# qui gère mieux beaucoup de spécificités de l'anglais (ex: contractions),
# puis le RegexpTokenizer pour 'découper' ensuite les adresses mails, URLS, etc...
# Il y aurait d'autres stratégies possibles, par exemples utiliser d'abord le module contractions.

def preprocess_1_tokenize(text, remove_punct=True):
    """
    Clean input text (lower, strip spaces), Tokenize, Remove punctuation (optional).

    Parameters:
    - text (str): Input text to tokenize.
    - remove_punct (bool): filter punctuation marks.

    Returns:
    - list: List of tokens.
    """
    # Nettoyage des majuscules et des espaces
    text = text.lower().strip()
    try:
        tokens = nltk.word_tokenize(text)
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(" ".join(tokens))  # Apply RegexpTokenizer to the entire list

        if remove_punct:
            # Remove punctuation (make sure)
            tokens = [token for token in tokens if token not in string.punctuation]

    except Exception as e:
        print(f"Error in tokenization: {e}")
        return []

    return tokens

# Apply to 'body' feature
raw_questions_tags['body_tokens'] = raw_questions_tags['body'].parallel_apply(
    lambda x: preprocess_1_tokenize(x))


In [ ]:
# Idem for 'title' column
raw_questions_tags['title_tokens'] = raw_questions_tags['title'].parallel_apply(
    lambda x: preprocess_1_tokenize(x))


In [ ]:
raw_questions_tags['nb_tokens_body'] = raw_questions_tags['body_tokens'].apply(len)
raw_questions_tags['nb_tokens_title'] = raw_questions_tags['title_tokens'].apply(len)

raw_questions_tags.describe()

# La question la + courte est composée de 7 tokens.
# La + longue, de presque 5000, soit... environ 15 pages ?!

display(raw_questions_tags.loc[(raw_questions_tags['nb_tokens_body'] == 7)
                               | (raw_questions_tags['nb_tokens_body'] == 4845), : ])

print(raw_questions_tags.loc[raw_questions_tags['nb_tokens_body'] == 4845, ['body']].values[0])
# Ca a l'air compliqué en effet !


### 2.4 Fréquence des mots


In [ ]:
# Extract tokens from the 'tokens_uniques' column
all_tokens = ([token for tokens_list in raw_questions_tags['title_tokens'] for token in tokens_list]
                 + [token for tokens_list in raw_questions_tags['body_tokens'] for token in tokens_list])

print(len(all_tokens))
# + d'un million de "mots" (tokens) différents !

# Calculate token frequencies using a loop
# J'ai découvert plus tard la fonction FreqDist() de nltk, qui fait la mm chose
token_frequencies_dict = {}
for token in all_tokens:
    token_frequencies_dict[token] = token_frequencies_dict.get(token, 0) + 1

# Display the first 50 items in the token frequencies dictionary
for token, frequency in list(token_frequencies_dict.items())[:50]:
    print(f"{token}: {frequency}")


In [ ]:
# Sort the dictionary items by values in descending order
sorted_token_frequencies = dict(sorted(token_frequencies_dict.items(), key=lambda item: item[1], reverse=True))

# Display the first 50 items in the token frequencies dictionary
for token, frequency in list(sorted_token_frequencies.items())[:50]:
    print(f"{token}: {frequency}")

# La plupart de ces mots appartiennent probablement à la liste des stopwords ?
# Note du futur : c plutôt l'inverse en fait, la liste des stopwords étant très courte (179 mots).


In [ ]:
# Visualisons

df_freq = pd.DataFrame(list(token_frequencies_dict.items()), columns=['Token', 'Frequency'])

# Sort the DataFrame by frequency in descending order
df_freq = df_freq.sort_values(by='Frequency', ascending=False)


In [ ]:
# Plot the distribution using Plotly

df = df_freq[:200]

fig = px.bar(df, x='Token', y='Frequency', labels={'Token': 'Token', 'Frequency': 'Frequency'},
             title='Token Frequency Distribution', text='Frequency')
fig.update_xaxes(tickangle=90, tickmode='array', tickvals=df.index, ticktext=df['Token'])

# Show the plot
fig.show()


In [ ]:
# Mots rares

# Count tags with frequency less than 10
tags_below_10 = sum(1 for frequency in sorted_token_frequencies.values() if frequency < 10)
print(tags_below_10)

# voire tres rares
# hapax (1 seule occurence)

# Count tags with frequency less than 2
tags_below_2 = sum(1 for frequency in sorted_token_frequencies.values() if frequency < 2)
print(tags_below_2)

# Environ 10% des tokens sont des hapax, ils n'apparaissent qu'une seule fois.
# Ces mots ne peuvent donc pas aider à la compréhension par le contexte.

# + de 20% des tokens apparaissent - de 10 fois ds tt le corpus.

tokens_hapax = [token for token, frequency in list(sorted_token_frequencies.items()) if frequency == 1]
tokens_rares = [token for token, frequency in list(sorted_token_frequencies.items()) if frequency <= 10]


### 2.5 Stopwords


In [ ]:
# Get a set of English stopwords from NLTK
stopwords = set(nltk.corpus.stopwords.words('english'))
print(len(stopwords), '\n')

important_tokens = [token for token in all_tokens if token not in stopwords]
print(len(important_tokens), '\n')
# + d'un quart de nos tokens appartiennent à la liste de stopwords (179 mots) !

# Display the first 50 items in the token frequencies dictionary
for token, frequency in list(sorted_token_frequencies.items())[:200]:
    if token in important_tokens:
        print(f"{token}: {frequency}")

# Il y a encore bcp de mots très communs. Pas sûr où placer le seuil.
# Le barplot était + parlant.


In [ ]:
# About these thresholds :

# lim_max could be a bit lower, maybe around 35 000
# But some important words appear between 35 000 and 50 000
# ex: java, the second most frequent tag (see 3.3)

# lim_min could probably be higher
# ask mentor

def get_forbidden_tokens(lim_max=50000, lim_min=1):
    too_frequent_tokens = [token for token, frequency in list(sorted_token_frequencies.items()) if frequency >= lim_max]
    too_rare_tokens = [token for token, frequency in list(sorted_token_frequencies.items()) if frequency <= lim_min]

    forbidden = set(stopwords)
    forbidden.update(too_frequent_tokens)
    forbidden.update(too_rare_tokens)

    return forbidden

forbidden = get_forbidden_tokens()


In [ ]:
def preprocess_2_remove_stopwords_frequent_rare(liste_tokens):
    filtered_list = [token for token in liste_tokens if token not in forbidden]

    return filtered_list

# Apply the preprocessing function to bodies
raw_questions_tags['body_tokens_no_stopwords'] = raw_questions_tags['body_tokens'].parallel_apply(
    preprocess_2_remove_stopwords_frequent_rare)


In [ ]:
# and titles
raw_questions_tags['title_tokens_no_stopwords'] = raw_questions_tags['title_tokens'].parallel_apply(
    preprocess_2_remove_stopwords_frequent_rare)


In [ ]:
raw_questions_tags['nb_tokens_body_no_stopwords'] = raw_questions_tags['body_tokens_no_stopwords'].apply(len)
raw_questions_tags['nb_tokens_title_no_stopwords'] = raw_questions_tags['title_tokens_no_stopwords'].apply(len)

raw_questions_tags.describe()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting 'nb_mots'
raw_questions_tags['nb_tokens_body'].hist(density=False, bins=50, ax=ax, color='#7cf', alpha=0.9, label='nb de mots / question')

# Plotting 'nb_mots non stopwords'
raw_questions_tags['nb_tokens_body_no_stopwords'].hist(density=False, bins=50, color='#4f6', ax=ax,
                                                          alpha=0.6, label='nb mots hors stopwords + freq')

plt.title('Vocabulaire', pad=20, fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

ax.legend()

plt.show()


### 2.6 Lemmatization + filtrage syntaxique


In [ ]:
# + filtrage implicite du vocabulaire non-anglais

# Lemmatization tends to be more accurate but can be slower than stemming.
# On a utilisé le stemming en français, dans le cours openclassrooms
# Utilisons la lemmatisation ici, + appropriée pour notre objectif
# (extraire le sens important, le thème, pour proposer des tags)
# On peut aussi profiter de l'analyse grammaticale (POS) pour
# conserver uniquement les mots les plus "importants" dans la phrase, les noms et les verbes.


def preprocess_3_lemmatize(tokens, filter=True):
    lemmatizer = WordNetLemmatizer()

    def get_wordnet_pos(pos_tag):
        if pos_tag.startswith('J'):
            return wordnet.ADJ
        elif pos_tag.startswith('R'):
            return wordnet.ADV
        elif pos_tag.startswith('V'):
            return wordnet.VERB
        elif pos_tag.startswith('N'):
            return wordnet.NOUN
        else:
            return wordnet.NOUN  # Default to noun if the part of speech is not recognized

    # Get part of speech for each token
    pos_tags = nltk.pos_tag(tokens)

    # Lemmatize each token
    lemmatized_tokens = []
    for token, pos_tag in pos_tags:
        if filter and pos_tag.startswith(('J', 'R')):
            # Exclude adjectives (J) and adverbs (R) if filter is enabled
            # = do nothing
            pass
            # = continue
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos=get_wordnet_pos(pos_tag)))

    return lemmatized_tokens


# Assuming raw_questions_tags is a DataFrame with a column 'tokens_uniques_no_stopwords'
# Replace 'tokens_uniques_no_stopwords' with the actual column name if needed
raw_questions_tags['body_lemms'] = raw_questions_tags['body_tokens_no_stopwords'].parallel_apply(preprocess_3_lemmatize)


In [ ]:
raw_questions_tags['title_lemms'] = raw_questions_tags['title_tokens_no_stopwords'].parallel_apply(preprocess_3_lemmatize)


In [ ]:
raw_questions_tags['nb_lemms_body'] = raw_questions_tags['body_lemms'].apply(len)
raw_questions_tags['nb_lemms_title'] = raw_questions_tags['title_lemms'].apply(len)


### 2.7 tokens (lemms) uniques


In [ ]:
# On aurait pu procéder à cette étape bien plus tôt (juste après l'étude de fréquences)
# et ainsi économiser du temps de calcul et de l'espace mémoire, mais on se garde ainsi
# la possibilité de tester notre LDA (notebook 3) sur un bag-of-words "classique".

def preprocess_4_keep_uniques_only(liste_tokens):
    seen_tokens = set()
    unique_tokens = []

    for token in liste_tokens:
        if token not in seen_tokens:
            seen_tokens.add(token)
            unique_tokens.append(token)

    return unique_tokens

# Apply the preprocessing function to body
raw_questions_tags['body_lemms_uniques'] = raw_questions_tags['body_lemms'].parallel_apply(preprocess_4_keep_uniques_only)


In [ ]:
# and title
raw_questions_tags['title_lemms_uniques'] = raw_questions_tags['title_lemms'].parallel_apply(preprocess_4_keep_uniques_only)


In [ ]:
raw_questions_tags['nb_lemms_uniques_body'] = raw_questions_tags['body_lemms_uniques'].apply(len)
raw_questions_tags['nb_lemms_uniques_title'] = raw_questions_tags['title_lemms_uniques'].apply(len)

display(raw_questions_tags.describe())

display(raw_questions_tags)

# Légère baisse pour les titres, en revanche
# le nb de tokens dans les questions (body) est divisé par 2 !


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting 'nb_mots'
raw_questions_tags['nb_tokens_body'].hist(density=False, bins=50, ax=ax, color='#7cf', alpha=0.9, label='nb de mots / question')

# Plotting 'nb_mots non stopwords'
raw_questions_tags['nb_tokens_body_no_stopwords'].hist(density=False, bins=50, color='#4f6', ax=ax,
                                                          alpha=0.6, label='nb mots hors stopwords + freq')

# Plotting 'lemms'
raw_questions_tags['nb_lemms_body'].hist(density=False, bins=50, color='#eb1', ax=ax,
                                                          alpha=1.0, label='nb_lemms')

# Plotting 'lemms uniques'
raw_questions_tags['nb_lemms_uniques_body'].hist(density=False, bins=50, color='#f55', ax=ax,
                                                          alpha=1.0, label='nb_lemms_uniques')

plt.title('Vocabulaire', pad=20, fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

ax.legend()

plt.show()

# On voit une petite différence !


In [ ]:
# Extract tokens from the 'tokens_uniques' column
all_lemms = set(([token for tokens_list in raw_questions_tags['title_lemms_uniques'] for token in tokens_list]
                 + [token for tokens_list in raw_questions_tags['body_lemms_uniques'] for token in tokens_list]))

print(len(all_lemms))
# Notre corpus a été divisé par 8 environ.


In [ ]:
# Corpus de départ
graphs_analyse_uni(raw_questions_tags, 'nb_tokens_body', bins=50, r=2, density=True)


In [ ]:
# Corpus d'arrivée
graphs_analyse_uni(raw_questions_tags, 'nb_lemms_uniques_body', bins=50, r=2, density=True)

# Moyenne divisée par 4


### 2.8 Explorer le corpus obtenu


In [ ]:
# Affichons 20 questions au hasard (+ les tags)
display(raw_questions_tags.sample(20))


In [ ]:
# Pistes restantes

# Le preprocessing de nos questions est assez satisfaisant, il répond à nos objectifs.
# Pour aller + loin, on pourrait par exemple examiner plus attentivement les valeurs numériques,
# les dates, URLs, adresses Email, noms propres, etc...

# Ces traitements plus avancés peuvent vite devenir horriblement compliqués
# avec un pur traitement nltk + regex, sans être jamais complètement satisfaisants.
# Filtrer les dates par exemple, juste les dates... C'est déjà un travail colossal à cause des
# (très) nombreux formats possibles.
# Identifier les noms propres... c'est simplement impossible sans machine learning.

# Nous allons donc en profiter pour tester spacy, qui est orienté objet et utilise différents modèles
# préentrainés. (cf partie 4)

# Avant cela, intéressons-nous aux tags, qui sont nos futures targets.


## 3 EDA et preprocessing des targets


### 3.1 Convertion (string to list)


In [ ]:
def convert_tags_string_to_list(tags_string):
    # just in case
    tags_string = tags_string.lower()
    # Split tags by '><'
    tags_list = tags_string.split('><')

    # Remove angle brackets from the first and last tag
    tags_list[0] = tags_list[0][1:]
    tags_list[-1] = tags_list[-1][:-1]

    return tags_list

# Apply the preprocessing function to the 'tags' column
raw_questions_tags['tags_list'] = raw_questions_tags['tags'].apply(convert_tags_string_to_list)

display(raw_questions_tags[['tags', 'tags_list']].head(10))


In [ ]:
raw_questions_tags['nb_tags'] = raw_questions_tags['tags_list'].apply(len)
raw_questions_tags['nb_tags'].describe()

# min : 5
# max : 6
# Un intervalle plutôt réduit.


### 3.2 Corpus des tags, Doublons éventuels


In [ ]:
all_tags = [tag for tags in raw_questions_tags['tags_list'] for tag in tags]
print(len(all_tags))

# Au cas où il y aurait des doublons ds les tags
raw_questions_tags['tags_uniques'] = raw_questions_tags['tags_list'].apply(preprocess_4_keep_uniques_only)

# display(raw_questions_tags)

# Fréquence
# Factoriser ?
tag_frequencies_dict = {}
for tag in all_tags:
    tag_frequencies_dict[tag] = tag_frequencies_dict.get(tag, 0) + 1

# Sort the dictionary items by values in descending order
sorted_tag_frequencies = dict(sorted(tag_frequencies_dict.items(), key=lambda item: item[1], reverse=True))

# Display the first 50 items in the tag frequencies dictionary
for tag, frequency in list(sorted_tag_frequencies.items())[:50]:
    print(f"{tag}: {frequency}")

# 250 000 tags (différents) !
# Ca repond a la question : est-ce qu'un humain pourrait facilement faire cette tache.
# Il faudrait déjà connaitre ts les tags possibles... Pas évident.

# Le tag le plus présent est python
# Est-ce qu'il y a plus de codeurs en python qu'en javascript ?
# Peut-être. Pas sûr.
# Est-ce qu'on va sur stack overflow plus souvent quand on fait du python ?
# Je dirais que oui. Il y a tjs de nveaux modules à découvrir en python !

# Classement intéressant !


### 3.3 Frequences des tags


In [ ]:
# Extract tags and frequencies
tags = list(sorted_tag_frequencies.keys())
frequencies = list(sorted_tag_frequencies.values())

# Plot
plt.figure(figsize=(12, 6))
plt.bar(tags[:100], frequencies[:100], color='skyblue')
plt.xlabel('Tags')
plt.ylabel('Frequency')
plt.title('Top 50 Tag Frequencies')
plt.xticks(rotation=90, ha='right', fontsize=8)  # Rotate x-axis labels for better readability
plt.show()


### 3.4 Les 20 tags les + frequents


In [ ]:
# + d'un tiers des question concernent python, js ou java.

# Faisons un top 20
df_freq = pd.DataFrame({'Tag': tags, 'Frequency': frequencies})

# Display the DataFrame
display(df_freq.head(20))

# plot a pie
colors = generate_random_pastel_colors(20)

fig, ax = plt.subplots(figsize=(10, 10))

patches, texts, autotexts = plt.pie(x=df_freq['Frequency'][:20], autopct='%1.1f%%',
    startangle=-30, labels=df_freq['Tag'][:20], textprops={'fontsize':11, 'color':'#000'},
    labeldistance=1.25, pctdistance=0.85, colors=colors)

plt.title(
label='20 most frequent Tags',
fontdict={"fontsize":17},
pad=20
)

for text in texts:
    # text.set_fontweight('bold')
    text.set_horizontalalignment('center')

# Customize percent labels
for autotext in autotexts:
    autotext.set_horizontalalignment('center')
    autotext.set_fontstyle('italic')
    autotext.set_fontsize('10')

#draw circle
centre_circle = plt.Circle((0,0),0.7,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.show()


### 3.5 Tags rares


In [ ]:
# A l'inverse, certains tags sont rares

# Count tags with frequency less than 10
tags_below_10 = sum(1 for frequency in sorted_tag_frequencies.values() if frequency < 10)
print(tags_below_10)

# voire tres rares
# tags hapax (1 seule occurence)

# Count tags with frequency less than 2
tags_below_2 = sum(1 for frequency in sorted_tag_frequencies.values() if frequency < 2)
print(tags_below_2)

# On les supprime ?
# seuil ?


### 3.6 WordCloud


In [ ]:
def show_cloud(list):
    cloud = WordCloud(background_color='white',
                    stopwords=[],
                    max_words=50).generate(" ".join(list))
    plt.imshow(cloud)
    plt.axis('off')
    plt.show()


show_cloud(all_tags)
# Pourquoi python tout petit ?? C le tag le + frequent !


In [ ]:
# analyse multi
# Nous allons rapidement visualiser les wordclouds des 5 tags les + fréquents.

most_frequent_tags = tags[:5]
print(most_frequent_tags)

subsets = dict()

for t in most_frequent_tags:
    subsets[t] = [tag for tags in raw_questions_tags['tags_list'] for tag in tags if t in tags]


In [ ]:
for subset in subsets:
    print(f'Tag : {subset}')
    show_cloud(subsets[subset])


In [ ]:
# Sans les répétitions :

for subset in subsets:
    print(f'Tag : {subset}')
    show_cloud(set(subsets[subset]))

# C'est plus parlant !


## 4 Spacy


In [ ]:
nlp = spacy.load("en_core_web_sm")


### 4.1 Test : use spacy as tokenizer


In [ ]:
def tokenize_with_spacy(text):
    doc = nlp(text)

    return [token.text for token in doc]


# Apply to original 'title' column
raw_questions_tags['title_spacy_tokens'] = raw_questions_tags['title'].parallel_apply(
    lambda x: tokenize_with_spacy(x))


In [ ]:
display(raw_questions_tags.head())


### 4.2 Preprocessing with spacy


In [ ]:
# pour la suppression des stopwods et tokens très fréquents / très rares
forbidden.update(nlp.Defaults.stop_words)

def do_everything_with_spacy(text):
    doc = nlp(text)

    # Filter out punctuation tokens, and tokens that are not nouns or verbs
    nouns_and_verbs = [token for token in doc if not token.is_punct and token.pos_ in ('NOUN', 'VERB')]

    # Filter out common stop words from nouns and verbs
    important_tokens = [token.lemma_ for token in nouns_and_verbs if token.text.lower() not in forbidden]

    # Additional filters for dates or other criteria can be added
    # but we already filtered non english words implicitely, by keeping only nouns and verbs

    return important_tokens


# Apply to original 'title' column
raw_questions_tags['title_spacy'] = raw_questions_tags['title'].parallel_apply(
    lambda x: do_everything_with_spacy(x))

# + rapide que le simple tokenizer ??


In [ ]:
# Apply to original 'body' column (without html)
raw_questions_tags['body_spacy'] = raw_questions_tags['body'].parallel_apply(
    lambda x: do_everything_with_spacy(x))


In [ ]:
# keep only uniques
raw_questions_tags['body_lemms_uniques_spacy'] = raw_questions_tags['body_spacy'].parallel_apply(preprocess_4_keep_uniques_only)
raw_questions_tags['title_lemms_uniques_spacy'] = raw_questions_tags['title_spacy'].parallel_apply(preprocess_4_keep_uniques_only)


In [ ]:
raw_questions_tags['nb_lemms_uniques_body_spacy'] = raw_questions_tags['body_lemms_uniques_spacy'].apply(len)
raw_questions_tags['nb_lemms_uniques_title_spacy'] = raw_questions_tags['title_spacy'].apply(len)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting 'lemms uniques'
raw_questions_tags['nb_lemms_uniques_body'].hist(density=False, bins=50, color='#7cf', ax=ax,
                                                          alpha=1.0, label='nb_lemms_uniques (nltk)')

# Plotting 'lemms'
raw_questions_tags['nb_lemms_body'].hist(density=False, bins=50, color='#4f6', ax=ax,
                                                          alpha=1.0, label='nb_lemms_uniques (spacy)')

plt.title('Vocabulaire', pad=20, fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

ax.legend()

plt.show()

# On voit une petite différence !


## Normalisation
